# �� Preprocesamiento Master para Survival Analysis

**Versión:** v2 (Definitiva)  
**Entrada:** `./data/processed/clean_survey_data.csv`  
**Salida:** `train_final.parquet`, `test_final.parquet`

---

## Justificación Metodológica (Basada en el Corpus Científico)

### 1. Vectorización de Competencias Técnicas (52D)

> **"Para los vectores de habilidades, rechazamos variables agregadas y usamos competencias específicas (52D) siguiendo a Börner et al. (2018) y Jang (2015), capturando la varianza intra-clase."**

### 2. Imputación Estocástica Uniforme

> **"Para la variable tiempo, aplicamos Imputación Estocástica Uniforme (Lawless, 2003) para convertir los rangos de la encuesta (interval-censored) en variables continuas, evitando la colinealidad artificial."**

### 3. Definición del Evento

> **Definimos E=1 cuando el graduado obtiene empleo STEM con correspondencia ≥3 (Getie Ayaneh et al., 2020).**

---

In [1]:
# ==============================================================================
# CONFIGURACIÓN
# ==============================================================================
import pandas as pd
import numpy as np
import json
import re
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

DATA_CLEAN = Path("data/processed/clean_survey_data.csv")
DICT_PATH = Path("../diccionario_maestro_stem.json")
OUTPUT_DIR = Path("data/processed")
WINDOW_SIZE = 30.0

print("✅ Configuración cargada")

✅ Configuración cargada


In [2]:
try:
    from unidecode import unidecode
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import train_test_split
except ImportError:
    !pip install unidecode scikit-learn pyarrow -q
    from unidecode import unidecode
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import train_test_split

df = pd.read_csv(DATA_CLEAN)
with open(DICT_PATH, 'r', encoding='utf-8') as f:
    DICCIONARIO_MAESTRO = json.load(f)['competencias']

print(f"✅ Datos: {len(df)} registros | Competencias: {len(DICCIONARIO_MAESTRO)}")

✅ Datos: 380 registros | Competencias: 52


In [3]:
# ==============================================================================
# MAPEO DE COLUMNAS POR ÍNDICE (Estructura fija del CSV)
# ==============================================================================

# Columnas identificadas por posición
COL_EDAD = df.columns[0]           # [0] 1.1 Edad
COL_GENERO = df.columns[1]         # [1] 1.2 Género
HAB_COLS_RAW = df.columns[3:10].tolist()  # [3-9] Soft skills
COL_TRABAJA = df.columns[11]       # [11] 4.1 ¿Actualmente estás trabajando?
COL_TIEMPO_TRABAJO = df.columns[15]  # [15] 4.1.5 Tiempo trabajando
COL_CORRESPONDENCIA = df.columns[16]  # [16] 4.1.6 Correspondencia 1-4
COL_BUSCA = df.columns[30]         # [30] 4.1.12 ¿Buscando trabajo?
COL_TIEMPO_BUSCA = df.columns[31]  # [31] 4.1.13 Tiempo buscando
COL_ASIGNATURAS = df.columns[35]   # [35] 6.1 Asignaturas relevantes

print("✅ Columnas mapeadas por índice")
print(f"   Trabaja [{11}]: {df[COL_TRABAJA].value_counts().to_dict()}")
print(f"   Correspondencia [{16}]: valores 1-4")

✅ Columnas mapeadas por índice
   Trabaja [11]: {'Si': 209, 'No': 171}
   Correspondencia [16]: valores 1-4


---
## Imputación Estocástica Uniforme

In [4]:
def parse_interval(text):
    if pd.isna(text): return np.nan, np.nan
    text = str(text).lower()
    if "menos de 6" in text: return 0.5, 6.0
    elif "6 meses" in text and "1 año" in text: return 6.0, 12.0
    elif "1" in text and "2 años" in text: return 12.0, 24.0
    elif "más de 2" in text or "mas de 2" in text: return 24.0, 30.0
    return np.nan, np.nan

# Tenure (empleados)
lower_t, upper_t = zip(*df[COL_TIEMPO_TRABAJO].apply(parse_interval))
lower_t, upper_t = np.array(lower_t, float), np.array(upper_t, float)
mask = ~np.isnan(lower_t)
tenure = np.zeros(len(df))
tenure[mask] = np.random.uniform(lower_t[mask], upper_t[mask])
df['tenure_sim'] = tenure

# Búsqueda (desempleados)
lower_b, upper_b = zip(*df[COL_TIEMPO_BUSCA].apply(parse_interval))
lower_b, upper_b = np.array(lower_b, float), np.array(upper_b, float)
mask = ~np.isnan(lower_b)
search = np.zeros(len(df))
search[mask] = np.random.uniform(lower_b[mask], upper_b[mask])
df['search_sim'] = search

print(f"✅ Imputación: Tenure {df['tenure_sim'].nunique()} únicos, Search {df['search_sim'].nunique()} únicos")

✅ Imputación: Tenure 210 únicos, Search 163 únicos


In [5]:
# ==============================================================================
# CONSTRUCCIÓN DE EVENT Y DURATION
# ==============================================================================

df['event'] = 0
df['duration'] = np.nan

# Normalizar columnas
trabaja = df[COL_TRABAJA].astype(str).str.strip().str.lower() == 'si'
correspondencia = pd.to_numeric(df[COL_CORRESPONDENCIA], errors='coerce').fillna(0)
busca = df[COL_BUSCA].astype(str).str.strip().str.lower() == 'si'

print(f"Verificación: Trabaja=Si: {trabaja.sum()}, Correspondencia>=3: {(correspondencia>=3).sum()}")

# EVENTO: Trabaja + Correspondencia >= 3
mask_event = trabaja & (correspondencia >= 3)
df.loc[mask_event, 'event'] = 1
df.loc[mask_event, 'duration'] = (WINDOW_SIZE - df.loc[mask_event, 'tenure_sim']).clip(lower=0.1)

# CENSURADO: No trabaja pero busca
mask_censored = (~trabaja) & busca
df.loc[mask_censored, 'event'] = 0
df.loc[mask_censored, 'duration'] = df.loc[mask_censored, 'search_sim'].clip(lower=0.1)

# CENSURADO: Subempleo
mask_sub = trabaja & (correspondencia < 3)
df.loc[mask_sub, 'event'] = 0
df.loc[mask_sub, 'duration'] = WINDOW_SIZE

# Limpiar
df_clean = df.dropna(subset=['duration']).copy()
df_clean = df_clean[df_clean['duration'] > 0].copy()

print(f"\n✅ TARGET:")
print(f"   Eventos: {(df_clean['event']==1).sum()} ({100*(df_clean['event']==1).mean():.1f}%)")
print(f"   Censurados: {(df_clean['event']==0).sum()}")
print(f"   Duration únicos: {df_clean['duration'].nunique()}")

Verificación: Trabaja=Si: 209, Correspondencia>=3: 169

✅ TARGET:
   Eventos: 169 (45.6%)
   Censurados: 202
   Duration únicos: 332


In [6]:
def extract_likert(v):
    if pd.isna(v): return np.nan
    m = re.search(r'^(\d)', str(v))
    return int(m.group(1)) if m else np.nan

HAB_COLS = []
for i, col in enumerate(HAB_COLS_RAW):
    name = f"hab_{i+1}"
    df_clean[name] = df_clean[col].apply(extract_likert)
    HAB_COLS.append(name)

print(f"✅ {len(HAB_COLS)} Soft Skills extraídas")

✅ 7 Soft Skills extraídas


In [7]:
def match_skills(texto, dic):
    if pd.isna(texto): return []
    texto = unidecode(str(texto).lower())
    return list(set(k for k, v in dic.items() for a in v.get('aliases',[]) if unidecode(a.lower()) in texto))

SKILLS = list(DICCIONARIO_MAESTRO.keys())
TECH_COLS = [f"tech_{s}" for s in SKILLS]

for s in SKILLS:
    df_clean[f"tech_{s}"] = 0

for idx in df_clean.index:
    for s in match_skills(df_clean.loc[idx, COL_ASIGNATURAS], DICCIONARIO_MAESTRO):
        df_clean.loc[idx, f"tech_{s}"] = 1

print(f"✅ {len(TECH_COLS)} Tech Skills | Cobertura: {(df_clean[TECH_COLS].sum(1)>0).mean():.1%}")

✅ 52 Tech Skills | Cobertura: 96.2%


In [8]:
df_clean['genero_m'] = df_clean[COL_GENERO].astype(str).str.lower().str.contains('masculino').astype(int)
df_clean['edad'] = pd.to_numeric(df_clean[COL_EDAD], errors='coerce')

FEATURES = ['edad', 'genero_m'] + HAB_COLS + TECH_COLS
X = df_clean[FEATURES].copy()
y = df_clean[['event', 'duration']].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y['event'])

# Escalado
scaler = MinMaxScaler()
for c in HAB_COLS + ['edad']:
    med = X_train[c].median()
    X_train[c] = X_train[c].fillna(med)
    X_test[c] = X_test[c].fillna(med)

scaler.fit(X_train[HAB_COLS])
X_train[HAB_COLS] = scaler.transform(X_train[HAB_COLS])
X_test[HAB_COLS] = scaler.transform(X_test[HAB_COLS])

print(f"✅ Train: {len(X_train)} | Test: {len(X_test)}")

✅ Train: 296 | Test: 75


In [9]:
train = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
test = pd.concat([X_test.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)

train.to_parquet(OUTPUT_DIR / "train_final.parquet", index=False)
test.to_parquet(OUTPUT_DIR / "test_final.parquet", index=False)

print("=" * 60)
print("🚀 PREPROCESAMIENTO COMPLETADO")
print("=" * 60)
print(f"Train: {len(train)} | Test: {len(test)}")
print(f"Features: {len(FEATURES)}")
print(f"Duration únicos: {train['duration'].nunique()}")
print(f"Event rate: {train['event'].mean():.1%}")
print(f"Correlación E-D: {train[['event','duration']].corr().iloc[0,1]:.4f}")

🚀 PREPROCESAMIENTO COMPLETADO
Train: 296 | Test: 75
Features: 61
Duration únicos: 265
Event rate: 45.6%
Correlación E-D: 0.5040
